In [2]:
! pip install tensorflow==1.13.1
! pip install tensorflow-gpu==1.13.1
! pip install keras==2.1.2
! pip install imutils

    100% |████████████████████████████████| 92.5MB 520kB/s eta 0:00:01
    100% |████████████████████████████████| 112kB 39.3MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 29.0MB/s ta 0:00:01
    100% |████████████████████████████████| 3.2MB 15.9MB/s ta 0:00:01
    100% |████████████████████████████████| 368kB 40.2MB/s ta 0:00:01
    100% |████████████████████████████████| 2.7MB 19.8MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 31.5MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 38.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for termcolor ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Running setup.py bdist_wheel for absl-py ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/8e/28/49/fad4e7f0b9a1227708cbbee4487ac8558a7334849cb81c813d
Successfully built termcolor absl-py
You are using pip version 10.0.1, however version 20.0.2 is availab

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 307kB 22.3MB/s ta 0:00:01
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Running setup.py bdist_wheel for imutils ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/16/84/1f/bf88641293cda2c8be81a5c4b8ca973dd9125a6dc3767417fd
Successfully built imutils
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# ça prend trop de temps de charger les données d'image directement de S3 
# Ecrire directement la ligne de commande suivante pour charger les données de S3 au répertoire de SagaMaker  : 
# aws s3 cp --recursive s3://chau-testbucket/dataset/training_set/cats /home/ec2-user/SageMaker/cats # 

In [3]:
import keras
import boto3
import os
import cv2
import numpy as np
import pandas as pd
from imutils import paths
from keras.models import load_model, Model
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
import matplotlib.image as mpimg
%matplotlib inline
from keras.utils import np_utils
from keras.models import Sequential
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import keras_preprocessing.image.image_data_generator
from keras.preprocessing.image import ImageDataGenerator
from imutils import paths
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import metrics
import random 
import os
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
#from keras.layers.core import Convolution2D
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [4]:
from keras.applications.mobilenet import MobileNet

# Data préparation première dataset #

In [5]:
# Le nom de répertoire des images #
image_path = list(paths.list_images('/home/ec2-user/SageMaker/fashion_robe_2/images5/'))
len(image_path) # 4400 images totales dans l'ensemble de répertoire #

4400

In [6]:
#Charger les images #
img = []
for i in range(0,len(image_path),4): # Sauter 4 lignes pour prendre une seule photo de chaque répertoire #
# Parce que pour chaque type d'image, il y a 4 photos les memes (différent de taille) mais on a besoin qu'une seule photo #
    images = list(paths.list_images('/home/ec2-user/SageMaker/fashion_robe_2/images5/'))[i] #le nom des répertoires des images #
    img.append(images)

In [7]:
# Le nombre d'images #
len(img) # 1100 images #

1100

In [8]:
# Print le lien #
type(img)

list

In [9]:
# Convestir dataset to dataframe # 
name_data = pd.DataFrame(img,columns=['lien'])
# Extrat le nom des images  #
name_image = []
for ligne in name_data['lien']:
    name = ligne.split("/")[7]
    name_image.append(name)
# Créer une colonne dans le data
name_data["name_image"] = name_image

# Extract encore une fois #
name = []
for ligne in name_data["name_image"]:
    n = ligne.split("-")[0]
    name.append(n)
name_data["name"] = name

In [11]:
# Charger les features des images des robes # 
# Importer les données de json # 
import os, json
import pandas as pd
# Le nom des répertoires des images # 
path_to_json = '/home/ec2-user/SageMaker/fashion_robe_2/images5/info_robes'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
#print(json_files)

In [12]:
# Load plusieurs des fichers json et load dans un data_frame #
path_to_json = '/home/ec2-user/SageMaker/fashion_robe_2/images5/info_robes'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

jsons_data = pd.DataFrame(columns=['titre','description','matiere','composition','coupe',
                                   'genreArticle','categorie','motifCouleur','longueur','longueurDesManches'])

for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)
        titre = json_text['titre']
        description = json_text['description']
        matiere =json_text['matiere']
        composition = json_text['composition']
        coupe = json_text['coupe']
        genreArticle = json_text['genreArticle']
        categorie = json_text['categorie']
        motifCouleur = json_text['motifCouleur']
        longueur = json_text['longueur']
        longueurDesManches = json_text['longueurDesManches']
        # here I push a list of data into a pandas DataFrame at row given by 'index'
        jsons_data.loc[index] = [titre,description,matiere,composition,coupe,genreArticle,categorie,motifCouleur,longueur,longueurDesManches]

In [13]:
jsons_data["image"] = json_files

In [14]:
# Extrat le nom des images de fichier json  # 
nom_img = []
for ligne in jsons_data["image"]:
    n = ligne.split(".")[0]
    nom_img.append(n)
# Créer une colonne des nom d'image # 
jsons_data["nom_img"] = nom_img

In [15]:
# Print les premières lignes #
jsons_data.head()

,titre,description,matiere,composition,coupe,genreArticle,categorie,motifCouleur,longueur,longueurDesManches,image,nom_img
0,Robe fourreau,"Matière et entretien Composition: 95% viscose,...",Jersey,"95% viscose, 5% élasthanne",,FEMME,robes-t-shirt-femme,Rayures,"Courte, 98 cm en taille S",Manches courtes,ex429f02u.json,ex429f02u
1,JINX DRESS - Robe en jean,Matière et entretien Composition: 100% coton M...,Denim,100% coton,Normale,FEMME,robes-en-jean-femme,Couleur unie,"Longueur midi, 128 cm en taille S",Manche extra courte,ch621c02a.json,ch621c02a
2,MIT BINDEGÜRTEL - Robe en jersey,"Matière et entretien Composition: 95% viscose,...",,"95% viscose, 5% élasthanne",Normale,FEMME,robes-t-shirt-femme,Rayures,Jusqu'aux genoux,Manches courtes,es181h036.json,es181h036
3,PRINTED STRAP DRESS - Robe en jersey,Matière et entretien Composition: 100% viscose...,,100% viscose,Normale,FEMME,robes-t-shirt-femme,Imprimé,"Longueur midi, 110 cm en taille S",Bretelles fines,tob21c02u.json,tob21c02u
4,MIDI DRESS - Robe longue - black,Matière et entretien Composition: 100% polyami...,Côtelé,100% polyamide,Près du corps,FEMME,robes-foureau-femme,,"Longueur midi, 122 cm en taille 38","Manches longues, 66 cm en taille 38",to221c09o.json,to221c09o


In [16]:
# Jointure des tables #
name_data = name_data.merge(jsons_data,left_on="name",right_on="nom_img")

In [17]:
# Print 30 lignes #
name_data.head(n=30)

,lien,name_image,name,titre,description,matiere,composition,coupe,genreArticle,categorie,motifCouleur,longueur,longueurDesManches,image,nom_img
0,/home/ec2-user/SageMaker/fashion_robe_2/images...,bj721c0d3-q11,bj721c0d3,RACER SHEATH SOLID - Robe fourreau,"Matière et entretien Composition: 91% coton, 9...",Côtelé,"91% coton, 9% élasthanne",,FEMME,robes-foureau-femme,Couleur unie,"Jusqu'aux genoux, 112 cm en taille S",Sans manches,bj721c0d3.json,bj721c0d3
1,/home/ec2-user/SageMaker/fashion_robe_2/images...,dp521c1py-q11,dp521c1py,ZEBRA MIDI - Robe fourreau,"Matière et entretien Composition: 97% viscose,...",Jersey,"97% viscose, 3% élasthanne",Près du corps,FEMME,robes-foureau-femme,Imprimé animal,"Jusqu'aux genoux, 102 cm en taille 36","Manches longues, 57 cm en taille 36",dp521c1py.json,dp521c1py
2,/home/ec2-user/SageMaker/fashion_robe_2/images...,wh021c044-q11,wh021c044,CONFETTI HEART HELENA MIDI DRESS - Robe fourreau,Matière et entretien Composition: 100% viscose...,,100% viscose,Cintrée,FEMME,robes-foureau-femme,Imprimé,"Longueur midi, 123 cm en taille 36","Manches trois-quarts, 54 cm en taille 36",wh021c044.json,wh021c044
3,/home/ec2-user/SageMaker/fashion_robe_2/images...,ve121c1mu-k11,ve121c1mu,VMTAILOR DRESS - Robe fourreau,Matière et entretien Composition: 100% polyest...,Jersey,100% polyester,Près du corps,FEMME,robes-foureau-femme,Rayures,"Courte, 86 cm en taille 38","Manches longues, 61 cm en taille 38",ve121c1mu.json,ve121c1mu
4,/home/ec2-user/SageMaker/fashion_robe_2/images...,ev621c000-k11,ev621c000,Robe fourreau,Matière et entretien Composition: 72% polyamid...,,"72% polyamide, 28% élasthanne",Cintrée,FEMME,robes-foureau-femme,Couleur unie,Jusqu'aux genoux,Manches longues,ev621c000.json,ev621c000
5,/home/ec2-user/SageMaker/fashion_robe_2/images...,ev621c000-q11,ev621c000,Robe fourreau,Matière et entretien Composition: 72% polyamid...,,"72% polyamide, 28% élasthanne",Cintrée,FEMME,robes-foureau-femme,Couleur unie,Jusqu'aux genoux,Manches longues,ev621c000.json,ev621c000
6,/home/ec2-user/SageMaker/fashion_robe_2/images...,ev621c000-g11,ev621c000,Robe fourreau,Matière et entretien Composition: 72% polyamid...,,"72% polyamide, 28% élasthanne",Cintrée,FEMME,robes-foureau-femme,Couleur unie,Jusqu'aux genoux,Manches longues,ev621c000.json,ev621c000
7,/home/ec2-user/SageMaker/fashion_robe_2/images...,iv321c076-a11,iv321c076,TRUMPET SLEEVE DRESS - Robe fourreau,Matière et entretien Composition: 63% polyeste...,,"63% polyester, 31% viscose, 6% élasthanne",Cintrée,FEMME,robes-foureau-femme,Couleur unie,"Jusqu'aux genoux, 109 cm en taille 38","Manches trois-quarts, 46 cm en taille 38",iv321c076.json,iv321c076
8,/home/ec2-user/SageMaker/fashion_robe_2/images...,iv321c076-m11,iv321c076,TRUMPET SLEEVE DRESS - Robe fourreau,Matière et entretien Composition: 63% polyeste...,,"63% polyester, 31% viscose, 6% élasthanne",Cintrée,FEMME,robes-foureau-femme,Couleur unie,"Jusqu'aux genoux, 109 cm en taille 38","Manches trois-quarts, 46 cm en taille 38",iv321c076.json,iv321c076
9,/home/ec2-user/SageMaker/fashion_robe_2/images...,gom21c00g-q11,gom21c00g,LONG SLEEVE TURTLE NECK DRESS - Robe fourreau,"Matière et entretien Composition: 93% viscose,...",Jersey,"93% viscose, 7% élasthanne",,FEMME,robes-t-shirt-femme,Couleur unie,"Jusqu'aux genoux, 101 cm en taille S","Manches longues, 60 cm en taille S",gom21c00g.json,gom21c00g


In [18]:
# Label #
name_data["categorie"].value_counts()

robes-t-shirt-femme    637
robes-foureau-femme    323
robes-en-jean-femme    140
Name: categorie, dtype: int64

# Data preparation deuxième dataset # 

In [19]:
# Charger le deuxième dataset d'image # 
image_path_1 = list(paths.list_images('/home/ec2-user/SageMaker/fashion/images3/'))
len(image_path_1) # 4400 images totales dans l'ensemble de répertoire # 10614 images #

10614

In [20]:
img_1 = []
for i in range(0,len(image_path_1),4): # Sauter 4 lignes pour prendre une seule photo de chaque répertoire #
# Parce que pour chaque type d'image, il y a 4 photos les memes (différent de taille) mais on a besoin qu'une seule photo #
    images = list(paths.list_images('/home/ec2-user/SageMaker/fashion/images3/'))[i] #le nom des répertoires des images #
    img_1.append(images)

In [21]:
len(img_1) # 2654 images #

2654

In [22]:
# Convestir dataset to dataframe # 
name_data_1 = pd.DataFrame(img_1,columns=['lien'])
# Extrat le nom des images  #
name_image = []
for ligne in name_data_1['lien']:
    name = ligne.split("/")[7]
    name_image.append(name)
# Créer une colonne dans le data
name_data_1["name_image"] = name_image

# Extract encore une fois #
name = []
for ligne in name_data_1["name_image"]:
    n = ligne.split("-")[0]
    name.append(n)
name_data_1["name"] = name

In [23]:
# Charger les features des images des robes # 
# Importer les données de json # 
import os, json
import pandas as pd
# Le nom des répertoires des images # 
path_to_json = '/home/ec2-user/SageMaker/fashion/images3/info_robes'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
#print(json_files)

In [24]:
len(json_files) # 2536 lignes #

2536

In [25]:
# Load plusieurs des fichers json et load dans un data_frame #
path_to_json = '/home/ec2-user/SageMaker/fashion/images3/info_robes'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

jsons_data_1 = pd.DataFrame(columns=['titre','description','matiere','composition','coupe',
                                   'genreArticle','categorie','motifCouleur','longueur','longueurDesManches'])

for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)
        titre = json_text['titre']
        description = json_text['description']
        matiere =json_text['matiere']
        composition = json_text['composition']
        coupe = json_text['coupe']
        genreArticle = json_text['genreArticle']
        categorie = json_text['categorie']
        motifCouleur = json_text['motifCouleur']
        longueur = json_text['longueur']
        longueurDesManches = json_text['longueurDesManches']
        # here I push a list of data into a pandas DataFrame at row given by 'index'
        jsons_data_1.loc[index] = [titre,description,matiere,composition,coupe,genreArticle,categorie,motifCouleur,longueur,longueurDesManches]

In [23]:
jsons_data_1.head()

,titre,description,matiere,composition,coupe,genreArticle,categorie,motifCouleur,longueur,longueurDesManches
0,ONLKENDEL DRESS BELT - Robe d'été,Matière et entretien Composition: 100% polyest...,Mousseline,100% polyester,Cintrée,FEMME,robes-ete-femme,À pois,"Longueur midi, 118 cm en taille 38","Manches longues, 63 cm en taille 38"
1,ABITO ALL IN ONE - Robe chemise,Matière et entretien Composition: 88% polyeste...,Mousseline,"88% polyester, 12% élasthanne, 100% polyester",Cintrée,FEMME,robes-chemise-femme,,"Extra courte, 85 cm en taille 38",62 cm en taille 38
2,Robe chemise,Matière et entretien Composition: 95% polyeste...,,"95% polyester, 5% élasthanne",Normale,FEMME,robes-chemise-femme,Couleur unie,"Courte, 97 cm en taille M",Manches longues
3,PAPILLON - Robe d'été,Matière et entretien Composition: 93% polyeste...,Mesh,"93% polyester, 7% élasthanne",,FEMME,robes-ete-femme,Floral,"Courte, 96 cm en taille M",Sans manches
4,Robe chemise,Matière et entretien Composition: 100% lyocell...,,100% lyocell,Ample,FEMME,robes-chemise-femme,Camouflage,"Jusqu'aux genoux, 96 cm en taille S","Manches trois-quarts, 48 cm en taille S"


In [24]:
jsons_data_1["image"] = json_files

In [25]:
# Extrat le nom des images de fichier json  # 
nom_img = []
for ligne in jsons_data_1["image"]:
    n = ligne.split(".")[0]
    nom_img.append(n)
# Créer une colonne des nom d'image # 
jsons_data_1["nom_img"] = nom_img

In [26]:
# Print les premières lignes #
jsons_data_1.head()

,titre,description,matiere,composition,coupe,genreArticle,categorie,motifCouleur,longueur,longueurDesManches,image,nom_img
0,ONLKENDEL DRESS BELT - Robe d'été,Matière et entretien Composition: 100% polyest...,Mousseline,100% polyester,Cintrée,FEMME,robes-ete-femme,À pois,"Longueur midi, 118 cm en taille 38","Manches longues, 63 cm en taille 38",on321c1mb.json,on321c1mb
1,ABITO ALL IN ONE - Robe chemise,Matière et entretien Composition: 88% polyeste...,Mousseline,"88% polyester, 12% élasthanne, 100% polyester",Cintrée,FEMME,robes-chemise-femme,,"Extra courte, 85 cm en taille 38",62 cm en taille 38,li621c036.json,li621c036
2,Robe chemise,Matière et entretien Composition: 95% polyeste...,,"95% polyester, 5% élasthanne",Normale,FEMME,robes-chemise-femme,Couleur unie,"Courte, 97 cm en taille M",Manches longues,jy121c0ck.json,jy121c0ck
3,PAPILLON - Robe d'été,Matière et entretien Composition: 93% polyeste...,Mesh,"93% polyester, 7% élasthanne",,FEMME,robes-ete-femme,Floral,"Courte, 96 cm en taille M",Sans manches,de121c0mm.json,de121c0mm
4,Robe chemise,Matière et entretien Composition: 100% lyocell...,,100% lyocell,Ample,FEMME,robes-chemise-femme,Camouflage,"Jusqu'aux genoux, 96 cm en taille S","Manches trois-quarts, 48 cm en taille S",tr121c008.json,tr121c008


In [34]:
# taille des fichers json #
jsons_data_1.shape # 2536 lignes #

(2536, 12)

In [35]:
# tailles des nombres des images  # 
name_data_1.shape # 2654 lignes # 

(2654, 3)

In [27]:
# Jointure des tables #
name_data_1 = name_data_1.merge(jsons_data_1,left_on="name",right_on="nom_img")

In [28]:
name_data_1.shape # 2653 lignes #

(2653, 15)

In [29]:
len(img_1)

2654

In [40]:
name_data_1["categorie"].value_counts()

robes-ete-femme          599
robes-de-soiree-femme    596
robes-longues-femme      593
robes-chemise-femme      445
robes-pull-femme         420
Name: categorie, dtype: int64

In [30]:
# Round bind les 2 datasets # 
df = pd.concat([name_data, name_data_1])

In [31]:
df.shape

(3753, 15)

In [32]:
df.head()

,lien,name_image,name,titre,description,matiere,composition,coupe,genreArticle,categorie,motifCouleur,longueur,longueurDesManches,image,nom_img
0,/home/ec2-user/SageMaker/fashion_robe_2/images...,bj721c0d3-q11,bj721c0d3,RACER SHEATH SOLID - Robe fourreau,"Matière et entretien Composition: 91% coton, 9...",Côtelé,"91% coton, 9% élasthanne",,FEMME,robes-foureau-femme,Couleur unie,"Jusqu'aux genoux, 112 cm en taille S",Sans manches,bj721c0d3.json,bj721c0d3
1,/home/ec2-user/SageMaker/fashion_robe_2/images...,dp521c1py-q11,dp521c1py,ZEBRA MIDI - Robe fourreau,"Matière et entretien Composition: 97% viscose,...",Jersey,"97% viscose, 3% élasthanne",Près du corps,FEMME,robes-foureau-femme,Imprimé animal,"Jusqu'aux genoux, 102 cm en taille 36","Manches longues, 57 cm en taille 36",dp521c1py.json,dp521c1py
2,/home/ec2-user/SageMaker/fashion_robe_2/images...,wh021c044-q11,wh021c044,CONFETTI HEART HELENA MIDI DRESS - Robe fourreau,Matière et entretien Composition: 100% viscose...,,100% viscose,Cintrée,FEMME,robes-foureau-femme,Imprimé,"Longueur midi, 123 cm en taille 36","Manches trois-quarts, 54 cm en taille 36",wh021c044.json,wh021c044
3,/home/ec2-user/SageMaker/fashion_robe_2/images...,ve121c1mu-k11,ve121c1mu,VMTAILOR DRESS - Robe fourreau,Matière et entretien Composition: 100% polyest...,Jersey,100% polyester,Près du corps,FEMME,robes-foureau-femme,Rayures,"Courte, 86 cm en taille 38","Manches longues, 61 cm en taille 38",ve121c1mu.json,ve121c1mu
4,/home/ec2-user/SageMaker/fashion_robe_2/images...,ev621c000-k11,ev621c000,Robe fourreau,Matière et entretien Composition: 72% polyamid...,,"72% polyamide, 28% élasthanne",Cintrée,FEMME,robes-foureau-femme,Couleur unie,Jusqu'aux genoux,Manches longues,ev621c000.json,ev621c000


In [33]:
# Split le string de longue  #
long = []
for ligne in df["longueur"]:
    long.append(ligne.split(",")[0])
df["long"] = long

In [34]:
# Boucle pour corriger le type de longuere #
test = []
for ligne in df['long']:
    if ligne.find('7/8') != (-1):
        ligne = "Longueur midi"
    elif ligne == '86 cm en taille S' or ligne =='91 cm en taille S' or ligne =='97 cm en taille 38' or ligne == '93 cm en taille 44' or ligne == '92 cm en taille M' or ligne == '85 cm en taille 38' or ligne == '93 cm en taille M' or ligne == '60 cm en taille S' or ligne == '96 cm en taille 38' or ligne == '95 cm en taille M' or ligne == '83 cm en taille S' or ligne == '76 cm en taille 36' or ligne == '95 cm en taille 38' :
        ligne = "Courte"
    elif ligne == '100 cm en taille 36':
        ligne = "Jusqu'aux genoux"
    elif ligne == '145 cm en taille 36' or ligne =='135 cm en taille 38' or ligne =='128 cm en taille S' or ligne =='148 cm en taille 38': 
        ligne = "Longue"
    elif ligne == 'Extra longue':
        ligne = 'Longue'
    elif ligne == 'Longueur mi-cuisse' or ligne == 'Normale':
        ligne = "Courte"
    test.append(ligne)
# Créer le colonne longeure #
df['long'] = test

In [35]:
# Examiner les catérogies #
df['long'].value_counts()

Jusqu'aux genoux    1029
Courte               877
Longueur midi        766
Longue               661
Extra courte         351
                      69
Name: long, dtype: int64

In [36]:
manche = []
for ligne in df['longueurDesManches']:
    n = ligne.split(",")[0]
    manche.append(n)
df['longueurDesManches'] = manche

In [37]:
df['longueurDesManches'].value_counts()

Manches longues         1108
Sans manches             686
Manches courtes          616
Manches trois-quarts     441
Manches mi-longues       243
Bretelles fines          192
Manche extra courte      190
Manche extra longue      144
                          77
Épaules nues              51
59 cm en taille 38         2
62 cm en taille 38         1
63 cm en taille M          1
57 cm en taille S          1
Name: longueurDesManches, dtype: int64

In [38]:
# Remplacer les string qui contient les valeurs numériques par les valeurs 0 #
df["longueurDesManches"] = np.where(~df.longueurDesManches.str[:2].str.isnumeric(),
                                        df.longueurDesManches.str.split(",").str[0],
                                        "")

In [39]:
# Fréquence des catérogues de long type #
df["longueurDesManches"].value_counts()

Manches longues         1108
Sans manches             686
Manches courtes          616
Manches trois-quarts     441
Manches mi-longues       243
Bretelles fines          192
Manche extra courte      190
Manche extra longue      144
                          82
Épaules nues              51
Name: longueurDesManches, dtype: int64

In [40]:
df.head()

,lien,name_image,name,titre,description,matiere,composition,coupe,genreArticle,categorie,motifCouleur,longueur,longueurDesManches,image,nom_img,long
0,/home/ec2-user/SageMaker/fashion_robe_2/images...,bj721c0d3-q11,bj721c0d3,RACER SHEATH SOLID - Robe fourreau,"Matière et entretien Composition: 91% coton, 9...",Côtelé,"91% coton, 9% élasthanne",,FEMME,robes-foureau-femme,Couleur unie,"Jusqu'aux genoux, 112 cm en taille S",Sans manches,bj721c0d3.json,bj721c0d3,Jusqu'aux genoux
1,/home/ec2-user/SageMaker/fashion_robe_2/images...,dp521c1py-q11,dp521c1py,ZEBRA MIDI - Robe fourreau,"Matière et entretien Composition: 97% viscose,...",Jersey,"97% viscose, 3% élasthanne",Près du corps,FEMME,robes-foureau-femme,Imprimé animal,"Jusqu'aux genoux, 102 cm en taille 36",Manches longues,dp521c1py.json,dp521c1py,Jusqu'aux genoux
2,/home/ec2-user/SageMaker/fashion_robe_2/images...,wh021c044-q11,wh021c044,CONFETTI HEART HELENA MIDI DRESS - Robe fourreau,Matière et entretien Composition: 100% viscose...,,100% viscose,Cintrée,FEMME,robes-foureau-femme,Imprimé,"Longueur midi, 123 cm en taille 36",Manches trois-quarts,wh021c044.json,wh021c044,Longueur midi
3,/home/ec2-user/SageMaker/fashion_robe_2/images...,ve121c1mu-k11,ve121c1mu,VMTAILOR DRESS - Robe fourreau,Matière et entretien Composition: 100% polyest...,Jersey,100% polyester,Près du corps,FEMME,robes-foureau-femme,Rayures,"Courte, 86 cm en taille 38",Manches longues,ve121c1mu.json,ve121c1mu,Courte
4,/home/ec2-user/SageMaker/fashion_robe_2/images...,ev621c000-k11,ev621c000,Robe fourreau,Matière et entretien Composition: 72% polyamid...,,"72% polyamide, 28% élasthanne",Cintrée,FEMME,robes-foureau-femme,Couleur unie,Jusqu'aux genoux,Manches longues,ev621c000.json,ev621c000,Jusqu'aux genoux


In [41]:
# La variable motifCouleur # 
df["motifCouleur"].value_counts()

Couleur unie         1741
Floral                511
                      414
Imprimé               269
Rayures               261
Chiné                 204
Imprimé animal        107
À pois                102
Carreaux               67
Paisley/cachemire      33
Multicolore            21
Chevrons                7
Camouflage              6
Fines rayures           5
Dégradé                 3
Dévoré                  2
Name: motifCouleur, dtype: int64

In [42]:
# La varible coupe # 
df["coupe"].value_counts()

Cintrée                       1414
Normale                        838
Près du corps                  660
                               584
Ample                          192
Cintrée, Asymétrique            25
Près du corps, Asymétrique      16
Normale, Asymétrique             7
Asymétrique                      5
Ample, Asymétrique               5
Oversize                         3
Regular Fit                      2
Slim                             2
Name: coupe, dtype: int64

In [43]:
# Boucle corriger 
coupetype = []
for ligne in df["coupe"]:
    coupe = ligne.split(",")[0]
    coupetype.append(coupe)
df["coupe_type"] = coupetype

In [44]:
df["coupe_type"].value_counts()

Cintrée          1439
Normale           845
Près du corps     676
                  584
Ample             197
Asymétrique         5
Oversize            3
Regular Fit         2
Slim                2
Name: coupe_type, dtype: int64

In [45]:
t = []
for ligne in df["coupe_type"]:
    if ligne == "Oversize":
        ligne = 'Ample'
    elif ligne == "Regular Fit" or ligne == "Slim":
        ligne = 'Près du corps'
    else:
        ligne = ligne
    t.append(ligne)
df["coupe_type"] = t

In [46]:
df["coupe_type"].value_counts()

Cintrée          1439
Normale           845
Près du corps     680
                  584
Ample             200
Asymétrique         5
Name: coupe_type, dtype: int64

In [47]:
# Fini data_processing end #
df.head()

,lien,name_image,name,titre,description,matiere,composition,coupe,genreArticle,categorie,motifCouleur,longueur,longueurDesManches,image,nom_img,long,coupe_type
0,/home/ec2-user/SageMaker/fashion_robe_2/images...,bj721c0d3-q11,bj721c0d3,RACER SHEATH SOLID - Robe fourreau,"Matière et entretien Composition: 91% coton, 9...",Côtelé,"91% coton, 9% élasthanne",,FEMME,robes-foureau-femme,Couleur unie,"Jusqu'aux genoux, 112 cm en taille S",Sans manches,bj721c0d3.json,bj721c0d3,Jusqu'aux genoux,
1,/home/ec2-user/SageMaker/fashion_robe_2/images...,dp521c1py-q11,dp521c1py,ZEBRA MIDI - Robe fourreau,"Matière et entretien Composition: 97% viscose,...",Jersey,"97% viscose, 3% élasthanne",Près du corps,FEMME,robes-foureau-femme,Imprimé animal,"Jusqu'aux genoux, 102 cm en taille 36",Manches longues,dp521c1py.json,dp521c1py,Jusqu'aux genoux,Près du corps
2,/home/ec2-user/SageMaker/fashion_robe_2/images...,wh021c044-q11,wh021c044,CONFETTI HEART HELENA MIDI DRESS - Robe fourreau,Matière et entretien Composition: 100% viscose...,,100% viscose,Cintrée,FEMME,robes-foureau-femme,Imprimé,"Longueur midi, 123 cm en taille 36",Manches trois-quarts,wh021c044.json,wh021c044,Longueur midi,Cintrée
3,/home/ec2-user/SageMaker/fashion_robe_2/images...,ve121c1mu-k11,ve121c1mu,VMTAILOR DRESS - Robe fourreau,Matière et entretien Composition: 100% polyest...,Jersey,100% polyester,Près du corps,FEMME,robes-foureau-femme,Rayures,"Courte, 86 cm en taille 38",Manches longues,ve121c1mu.json,ve121c1mu,Courte,Près du corps
4,/home/ec2-user/SageMaker/fashion_robe_2/images...,ev621c000-k11,ev621c000,Robe fourreau,Matière et entretien Composition: 72% polyamid...,,"72% polyamide, 28% élasthanne",Cintrée,FEMME,robes-foureau-femme,Couleur unie,Jusqu'aux genoux,Manches longues,ev621c000.json,ev621c000,Jusqu'aux genoux,Cintrée


In [48]:
df = df.drop("coupe",axis=1)

In [49]:
df.columns

Index(['lien', 'name_image', 'name', 'titre', 'description', 'matiere',
       'composition', 'genreArticle', 'categorie', 'motifCouleur', 'longueur',
       'longueurDesManches', 'image', 'nom_img', 'long', 'coupe_type'],
      dtype='object')

In [50]:
df.shape # 3753 ligne #

(3753, 16)

In [51]:
df1 = df[["lien","name","matiere","coupe_type","motifCouleur","longueurDesManches","categorie","long"]]

In [52]:
df1.head()

,lien,name,matiere,coupe_type,motifCouleur,longueurDesManches,categorie,long
0,/home/ec2-user/SageMaker/fashion_robe_2/images...,bj721c0d3,Côtelé,,Couleur unie,Sans manches,robes-foureau-femme,Jusqu'aux genoux
1,/home/ec2-user/SageMaker/fashion_robe_2/images...,dp521c1py,Jersey,Près du corps,Imprimé animal,Manches longues,robes-foureau-femme,Jusqu'aux genoux
2,/home/ec2-user/SageMaker/fashion_robe_2/images...,wh021c044,,Cintrée,Imprimé,Manches trois-quarts,robes-foureau-femme,Longueur midi
3,/home/ec2-user/SageMaker/fashion_robe_2/images...,ve121c1mu,Jersey,Près du corps,Rayures,Manches longues,robes-foureau-femme,Courte
4,/home/ec2-user/SageMaker/fashion_robe_2/images...,ev621c000,,Cintrée,Couleur unie,Manches longues,robes-foureau-femme,Jusqu'aux genoux


In [53]:
# Exporter le fichier to _csv #
df1.to_csv("data.csv")